Code created by Rafael Solar in Google colab.


This program includes the training of a VGG16 convolutional network using a dataset containing five animal classes.

The code is based on the content of Section 8 of the book F. Chollet, Python Deep Learning, Manning, 2nd ed., 2021.

In [ ]:
#Connect to our files in Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import the necessary libraries.
import pathlib
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
# Using the file path, we process the dataset into three groups: "train," "validation (val)," and "test."
# Each group contains 5 classes: cat, dog, chicken, horse, and spider.

path_drive_dataset = '/content/drive/MyDrive/Rafael_Solar_Hdez-Tesis-Lic/Programas/Modelo_Clasificacion-otono2022/dataset_animal'
new_base_dir = pathlib.Path(path_drive_dataset)
train_dataset = image_dataset_from_directory(
    new_base_dir/'train',
    image_size=(180,180),
    batch_size=32)

val_dataset = image_dataset_from_directory(
    new_base_dir/'validation',
    image_size=(180,180),
    batch_size=32)

test_dataset = image_dataset_from_directory(
    new_base_dir/'test',
    image_size=(180,180),
    batch_size=32)

## Model implementing VGG16

In [ ]:
# Implementing the base instance of the VGG16 convolutional network, freezing all layers, and excluding the top or classifier.
conv_base_vgg = keras.applications.vgg16.VGG16(
    weights = 'imagenet',
    include_top = False,)

# "freeze" the layers of the pre-trained model that we will not fine-tune, while leaving the last 4 layers to be adjusted with the dataset.
conv_base_vgg.trainable = True
for layer in conv_base_vgg.layers[:-4]:
    layer.trainable = False

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
conv_base_vgg.summary() #VGG16 summary

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

## Construction of the model that will train the last layers and our classifier.

In [ ]:
# We apply the data augmentation technique with rotation and zoom parameters set to probabilities of 0.1 and 0.2.

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2)
    ]
)

# Structure of the "model_5animals".
# Image parameters: 180 pixels by 180 pixels, with 3 filters as they are RGB images.
inputs = keras.Input(shape=(180,180,3))
# Apply data augmentation to the input images.
x = data_augmentation(inputs)
# Preprocess the input image.
x = keras.applications.vgg16.preprocess_input(x)
# Implement the VGG convolutional base without the classifier.
x = conv_base_vgg(x)
# Transform the network to adapt the dimension to a 1xn vector.
x = layers.Flatten() (x)
# Create a dense layer with 256 neurons.
x = layers.Dense(256) (x)
# Use dropout with a probability of 0.5.
x = layers.Dropout(0.5) (x)
# Define the network output "5animals" with 5 neurons, corresponding to the network's classes.
outputs = layers.Dense(5, activation='softmax') (x)
#---------------------------------------------
# Construction of the "5animals" model
model_5animals = keras.Model(inputs, outputs)
#---------------------------------------------

# Model compilation with the loss function, optimizer, and metrics.
model_5animals.compile(loss = 'sparse_categorical_crossentropy',
              optimizer= keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics = ['accuracy'])

In [ ]:
model_5animals.summary() #Our neuronal network summary

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 sequential_3 (Sequential)   (None, 180, 180, 3)       0         
                                                                 
 tf.__operators__.getitem_3   (None, 180, 180, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_3 (TFOpLambd  (None, 180, 180, 3)      0         
 a)                                                              
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten_3 (Flatten)         (None, 12800)             0   

## Training the 5animals model.

In [ ]:
# Training of the 5animals neural network.

history = model_5animals.fit(
    train_dataset,
    epochs = 10,
    validation_data = val_dataset,
)


Epoch 1/10
157/157 [==============================] - 2803s 17s/step - loss: 5.6841 - accuracy: 0.6232 - val_loss: 0.8189 - val_accuracy: 0.8728
Epoch 2/10
157/157 [==============================] - 36s 228ms/step - loss: 1.0858 - accuracy: 0.7838 - val_loss: 0.5028 - val_accuracy: 0.9060
Epoch 3/10
157/157 [==============================] - 36s 225ms/step - loss: 0.6712 - accuracy: 0.8380 - val_loss: 0.3427 - val_accuracy: 0.9316
Epoch 4/10
157/157 [==============================] - 36s 225ms/step - loss: 0.5094 - accuracy: 0.8748 - val_loss: 0.2740 - val_accuracy: 0.9464
Epoch 5/10
157/157 [==============================] - 36s 228ms/step - loss: 0.3587 - accuracy: 0.8992 - val_loss: 0.2714 - val_accuracy: 0.9512
Epoch 6/10
157/157 [==============================] - 36s 225ms/step - loss: 0.2906 - accuracy: 0.9240 - val_loss: 0.2677 - val_accuracy: 0.9524
Epoch 7/10
157/157 [==============================] - 36s 224ms/step - loss: 0.2246 - accuracy: 0.9372 - val_loss: 0.2430 - val_ac

### Print the accuracy on the original training set and the animal testing set.

In [ ]:
test_loss, test_acc = model_5animals.evaluate(train_dataset) # training
print(f"Test accuracy using training set: {test_acc:.3f}")


157/157 [==============================] - 19s 115ms/step - loss: 0.0800 - accuracy: 0.9852
Test accuracy using training set: 0.985


In [ ]:
test_loss, test_acc = model_5animals.evaluate(test_dataset) #testing
print(f"Test accuracy using test set: {test_acc:.3f}")

157/157 [==============================] - 2096s 13s/step - loss: 0.2121 - accuracy: 0.9700
Test accuracy using test set: 0.970


### Save the trained model model_5animals.

In [ ]:
#Guardar el modelo 5animales entrenado
!mkdir -p saved_model
model_5animals.save('...models_save_path/model5animales.h5')